In [ ]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [ ]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [ ]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [ ]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [ ]:
# Indicator preparattion
config = FileUtil.load_logging_config()
logging_config = LoggingConfig(config)
logger = logging_config.logging_setup()

tx_curr_service = ComputeTxCurrService(logger)
tx_new_service = ComputeTxNewService(tx_curr_service, logger)

tx_curr_indicator_metadata_port = TxCurrIndicatorMetadataAdapter(api)
arv_dispense_indicator_metadata_port = ArvDispenseIndicatorMetadataAdapter(api)
tx_new_indicator_metadata_port = TxNewIndicatorMetadataAdapter(api)

tx_curr_disaggregation_service = ComputeTxCurrDisaggregationService(tx_curr_indicator_metadata_port, arv_dispense_indicator_metadata_port, logger)
tx_new_disaggretation_service = ComputeTxNewDisaggregationService(tx_new_indicator_metadata_port, logger)

tx_ml_indicator_metadata_port = TxMlIndicatorMetadataAdapter(api)
tx_ml_service = ComputeTxMlService(logger)
tx_ml_disaggregation_service = ComputeTxMlDisaggregationService(tx_ml_indicator_metadata_port)

patient_consultation_form = PatientConsultationForm(api)
consultation_port = ConsultationAdapter(patient_consultation_form)
patient_laboratoty_form = PatientLaboratoryForm(logger,api)
laboratory_port = LaboratoryAdapter(patient_laboratoty_form)

tx_pvls_denominator_service = ComputeTxPvlsDenominatorService(logger, tx_curr_service, laboratory_port, consultation_port)
tx_pvls_numerator_service = ComputeTxPvlsNumeratorService()

tx_pvls_denominator_indicator_metadata_port = TxPvlsDenominatorIndicatorMetadataAdapter(api)
tx_pvls_denominator_pred_breast_indicator_metadata_port = TxPvlsDenominatorPregBreastIndicatorMetadataAdapter(api)
tx_pvls_denominator_disaggregation_service = ComputeTxPvlsDenominatorDisaggregationService(tx_pvls_denominator_indicator_metadata_port, tx_pvls_denominator_pred_breast_indicator_metadata_port)

tx_pvls_numerator_indicator_metadata_port = TxPvlsNumeratorIndicatorMetadataAdapter(api)
tx_pvls_numerator_pred_breast_indicator_metadata_port = TxPvlsNumeratorPregBreastIndicatorMetadataAdapter(api)
tx_pvls_numerator_disaggregation_service = ComputeTxPvlsNumeratorDisaggregationService(tx_pvls_numerator_indicator_metadata_port, tx_pvls_numerator_pred_breast_indicator_metadata_port)

pharmacy_form = PatientPharmacyForm(logger, api)
pharmacy_port = PharmacyAdapter(pharmacy_form)
tx_rtt_service = ComputeTxRttService(logger, pharmacy_port, tx_curr_service)
tx_rtt_indicator_metadata_port = TxRttIndicatorMetadataAdapter(api)
tx_rtt_itt_indicator_metadata_port = TxRttIttIndicatorMetadataAdapter(api)
tx_rtt_disaggregation_service = ComputeTxRttDisaggregationService(logger, tx_rtt_indicator_metadata_port, tx_rtt_itt_indicator_metadata_port)

In [ ]:
# enrollments = pd.DataFrame(columns=['enrollment', 'trackedEntity', 'program', 'status', 'orgUnit', 'enrolledAt', 'patientIdentifier', 'patientAge', 'patientSex','patientName', 
#                                     'artStartDate', 'firstConsultationDate','pickupQuantity', 'lastPickupDate', 'nextPickupDate', 'lastCD4', 'viralLoadResultDate', 
#                                     'viralLoadResultValue', 'transferedOut', 'dateOfTransfer','dead', 'dateOfDeath'])

# enrollments.to_csv('ENROLLMENTS.csv', index=False)

In [ ]:
# # Load patients enrolled
# for org_unit in org_units:

#     org_unit = org_unit['id']

#     #get all patient enrollments
#     patients_enrolled = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': CARE_AND_TREATMENT, 'fields':'{,enrollment, enrolledAt, orgUnit, trackedEntity, program, status,}'})
#     patients_enrolled = patients_enrolled.json()['instances']

#     logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_enrolled)} enrolled patient(s)")
    
#     counter = 1

#     for patient_enrolled in patients_enrolled:
#         patient_id = patient_enrolled['trackedEntity']

#         patient_demographics = PatientDemographicForm(api)
#         patient_demographics.add_demographics(patient_enrolled)

#         patient_consultation= PatientConsultationForm(patient_id, org_unit, api)
#         patient_consultation.add_first_consultation(patient_enrolled)

#         patient_pharmacy_form =  PatientPharmacyForm(patient_id, org_unit, api)
#         patient_pharmacy_form.add_last_pharmacy(patient_enrolled, end_period)

#         patient_laboratory_from =  PatientLaboratoryForm(patient_id, org_unit, api)
#         patient_laboratory_from.add_laboratory(patient_enrolled)

#         patient_outcome_form =  PatientOutcomeForm(patient_id, org_unit, api)
#         patient_outcome_form.add_final_outcome(patient_enrolled)

#         logger.info(f"From {len (patients_enrolled)} patietnts enrolled, {counter} (is) are ready to be processed.")
#         counter = counter + 1 

#     if patients_enrolled:
#         patients_enrolled = pd.json_normalize(patients_enrolled)
        
#         enrollments = pd.read_csv('ENROLLMENTS.csv')
#         enrollments = pd.concat([enrollments, patients_enrolled])
#         enrollments.to_csv('ENROLLMENTS.csv', index=False, encoding='utf-8')

In [ ]:
enrollments = pd.read_csv('ENROLLMENTS.csv')
enrollments = enrollments.to_dict(orient='records')

tx_curr_patients = tx_curr_service.compute(enrollments, end_period)
tx_curr_patients_disaggregation = tx_curr_disaggregation_service.compute(tx_curr_patients, end_period)

tx_new_patients = tx_new_service.compute(tx_curr_patients, start_period, end_period)
tx_new_patients_disaggregation = tx_new_disaggretation_service.compute(tx_new_patients, end_period)

tx_ml_patients = tx_ml_service.compute(enrollments, start_period, end_period, ComputeTxMlService.QUARTERLY_DAYS_EXPECTED)
tx_ml_patients_disaggregation = tx_ml_disaggregation_service.compute(tx_ml_patients, end_period)

tx_pvls_denominator_patients = tx_pvls_denominator_service.compute(tx_curr_patients, end_period)
tx_pvls_denominator_patients_disaggregation = tx_pvls_denominator_disaggregation_service.compute(tx_pvls_denominator_patients, end_period)

tx_pvls_numerator_patients = tx_pvls_numerator_service.compute(tx_pvls_denominator_patients)
tx_pvls_numerator_patients_disaggregation = tx_pvls_numerator_disaggregation_service.compute(tx_pvls_numerator_patients, end_period)

tx_rtt_patients = tx_rtt_service.compute(enrollments, start_period, end_period)
tx_rtt_patients_disaggregation = tx_rtt_disaggregation_service.compute(tx_rtt_patients, end_period)

combination = tx_curr_patients_disaggregation + tx_new_patients_disaggregation + tx_ml_patients_disaggregation + tx_pvls_denominator_patients_disaggregation + tx_pvls_numerator_patients_disaggregation + tx_rtt_patients_disaggregation

if combination:
    # extract data
    indicators = pd.json_normalize(combination)
    indicators['period'] = period
    indicators = indicators[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
    indicators = indicators.sort_values(['orgUnit', 'dataElement', 'period'])
    indicators.to_csv('TX_DATA.csv', index=False)
